# Segmenting and Clustering Neighborhoods in Toronto
Week 3 capstone project - IBM Data Science

## TOC:
* [PART 1 : importing and cleaning data](#first-bullet)
* [PART 2 : Getting geospatial coordinates](#second-bullet)
* [PART 3 : Clusturing Neighborhoods](#third-bullet)

## PART 1 : importing and cleaning data <a class="anchor" id="first-bullet"></a>

### Installing/importing libs

In [1]:
# install lxml only if not already installed
%pip install lxml
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


### Scraping page

In [2]:
# scraping the html page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tor_neigh = pd.read_html(url)[0]

In [3]:
# displaying data as scraped
tor_neigh.head()

,Post Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Cleaning data
- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [4]:
# remove Not assigned Borough
tor_neigh_clean = tor_neigh[tor_neigh['Borough'] != 'Not assigned']
tor_neigh_clean.reset_index(drop=True, inplace=True)
tor_neigh_clean

,Post Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [5]:
# Wikipedia page shape seems to have been modified : 2 Neighborhood with same post code are on same line
# just check that we have same number of line and number of unique post code to be sure
print("number of line in dataset: ", tor_neigh_clean.shape[0])
print("number of unique values in postcode: ", len(tor_neigh_clean['Post Code'].unique()))

number of line in dataset:  103
number of unique values in postcode:  103


In [6]:
# Rename not assigned neighborhood as Borough
tor_neigh_clean.replace({'Neighborhood':'Not assigned'}, tor_neigh_clean[['Borough']], inplace=True)
tor_neigh_clean

,Post Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [7]:
# getting the shape
tor_neigh_clean.shape

(103, 3)

## PART 2 : Getting geospatial coordinates <a class="anchor" id="second-bullet"></a>

_Note : I tried with geocoder module but google requires an API key and OSM does not return enough values, so I used the file instead_
### First try with geocoder

In [8]:
# installing geocoder
%pip install geocoder
import geocoder

Note: you may need to restart the kernel to use updated packages.


In [9]:
# importing function from instructions
# using open street maps instead of google
def getCoordinates(postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.osm('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = (g.lat, g.lng)
    
    return lat_lng_coords 

In [10]:
loc_post_code = tor_neigh_clean['Post Code'].apply(getCoordinates)
loc_post_code
## TOO MANY EMPTY VALUES, I'LL USE THE FILE

0      (43.6534817, -79.3839347)
1                   (None, None)
2                   (None, None)
3                   (None, None)
4      (43.6534817, -79.3839347)
                 ...            
98                  (None, None)
99                  (None, None)
100                 (None, None)
101                 (None, None)
102                 (None, None)
Name: Post Code, Length: 103, dtype: object

### Using the provided csv file yo get coordinates

In [11]:
df_coord = pd.read_csv('https://cocl.us/Geospatial_data')
df_coord

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


### Adding coordinates to the data frame

In [12]:
df_coord.rename(columns={'Postal Code': 'Post Code'}, inplace=True)
toronto_data = tor_neigh_clean.merge(df_coord, on='Post Code')
toronto_data

,Post Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## PART 3 : Clusturing Neighborhoods <a class="anchor" id="third-bullet"></a>

### Importing libraries

In [13]:
#%pip install geopy # uncomment this line if you haven't installed geopy
#%pip install matplotlib # uncomment this line if you haven't installed matplotlib
#%pip install sklearn # uncomment this line if you haven't installed sklearn
#%pip install folium # uncomment this line if you haven't installed folium
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

import numpy as np

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library


### Checking data

In [14]:
toronto_data.head()

,Post Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Display a map of Neighborhood
First get coordinates of Toronto to center maps

In [15]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

Display a map of Neighborhood

In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Getting venues and organizing data for analysis
Add client ID and client secret for Foursquare requests

In [17]:
import os
CLIENT_ID = os.environ['FOURSQUARE_ID'] # your Foursquare ID
CLIENT_SECRET = os.environ['FOURSQUARE_SECRET'] # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500

Add function to get Venues

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

get Venues for each Neighborhood

In [19]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [20]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
print(toronto_venues['Venue Category'].unique())

There are 267 uniques categories.
['Park' 'Food & Drink Shop' 'Bus Stop' 'Hockey Arena'
 'Portuguese Restaurant' 'Coffee Shop' 'French Restaurant' 'Intersection'
 'Pizza Place' 'Bakery' 'Distribution Center' 'Spa' 'Restaurant' 'Pub'
 'Breakfast Spot' 'Gym / Fitness Center' 'Historic Site' 'Farmers Market'
 'Dessert Shop' 'Chocolate Shop' 'Performing Arts Venue'
 'Mexican Restaurant' 'Café' 'Yoga Studio' 'Theater' 'Event Space'
 'Asian Restaurant' 'Shoe Store' 'Ice Cream Shop' 'Electronics Store'
 'Art Gallery' 'Bank' 'Beer Store' 'Health Food Store' 'Wine Shop'
 'Antique Shop' 'Boutique' 'Furniture / Home Store'
 'Vietnamese Restaurant' 'Clothing Store' 'Accessories Store'
 'Miscellaneous Shop' 'Sporting Goods Shop' 'Creperie' 'Sushi Restaurant'
 'Arts & Crafts Store' 'Burrito Place' 'Beer Bar' 'Hobby Shop' 'Diner'
 'Fried Chicken Joint' 'Smoothie Shop' 'Sandwich Place' 'Gym'
 'College Auditorium' 'Bar' 'College Cafeteria' 'General Entertainment'
 'Fast Food Restaurant' 'Caribbean Rest

_One of the categories is names Neighborhood which will make a duplicate entree, rename this one to 'Neighborhood venue'_

In [21]:
toronto_venues.replace({'Venue Category': 'Neighborhood'}, 'Neighborhood venue', inplace=True)
print(toronto_venues['Venue Category'].unique())

['Park' 'Food & Drink Shop' 'Bus Stop' 'Hockey Arena'
 'Portuguese Restaurant' 'Coffee Shop' 'French Restaurant' 'Intersection'
 'Pizza Place' 'Bakery' 'Distribution Center' 'Spa' 'Restaurant' 'Pub'
 'Breakfast Spot' 'Gym / Fitness Center' 'Historic Site' 'Farmers Market'
 'Dessert Shop' 'Chocolate Shop' 'Performing Arts Venue'
 'Mexican Restaurant' 'Café' 'Yoga Studio' 'Theater' 'Event Space'
 'Asian Restaurant' 'Shoe Store' 'Ice Cream Shop' 'Electronics Store'
 'Art Gallery' 'Bank' 'Beer Store' 'Health Food Store' 'Wine Shop'
 'Antique Shop' 'Boutique' 'Furniture / Home Store'
 'Vietnamese Restaurant' 'Clothing Store' 'Accessories Store'
 'Miscellaneous Shop' 'Sporting Goods Shop' 'Creperie' 'Sushi Restaurant'
 'Arts & Crafts Store' 'Burrito Place' 'Beer Bar' 'Hobby Shop' 'Diner'
 'Fried Chicken Joint' 'Smoothie Shop' 'Sandwich Place' 'Gym'
 'College Auditorium' 'Bar' 'College Cafeteria' 'General Entertainment'
 'Fast Food Restaurant' 'Caribbean Restaurant' 'Japanese Restaurant'
 'Ga

In [22]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe at the beginning
toronto_onehot.insert(0,'Neighborhood', toronto_venues['Neighborhood'])

toronto_onehot.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [23]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').sum().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

### Displaying top 5 venues / Neighborhood

In [24]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0               Skating Rink   1.0
1                     Lounge   1.0
2             Breakfast Spot   1.0
3  Latin American Restaurant   1.0
4         Mexican Restaurant   0.0


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place   2.0
1  Skating Rink   1.0
2      Pharmacy   1.0
3   Coffee Shop   1.0
4           Pub   1.0


----Bathurst Manor, Wilson Heights, Downsview North----
              venue  freq
0       Coffee Shop   2.0
1              Bank   2.0
2       Pizza Place   1.0
3     Shopping Mall   1.0
4  Sushi Restaurant   1.0


----Bayview Village----
                 venue  freq
0                 Café   1.0
1                 Bank   1.0
2  Japanese Restaurant   1.0
3   Chinese Restaurant   1.0
4        Movie Theater   0.0


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place   2.0
1    Sushi Restaurant   2.0
2          Restaurant   2.0
3  Italian Restaurant   2.0
4        

                             venue  freq
0                   Baseball Field   1.0
1                Accessories Store   0.0
2        Middle Eastern Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----Humewood-Cedarvale----
               venue  freq
0              Field   1.0
1               Park   1.0
2              Trail   1.0
3       Hockey Arena   1.0
4  Martial Arts Dojo   0.0


----India Bazaar, The Beaches West----
                venue  freq
0      Sandwich Place   2.0
1                Park   2.0
2       Burrito Place   1.0
3          Steakhouse   1.0
4  Light Rail Station   1.0


----Kennedy Park, Ionview, East Birchmount Park----
              venue  freq
0  Department Store   1.0
1    Discount Store   1.0
2       Bus Station   1.0
3        Hobby Shop   1.0
4       Coffee Shop   1.0


----Kensington Market, Chinatown, Grange Park----
                           venue  freq
0                           Café   5.0
1             

                       venue  freq
0                       Park   1.0
1          Convenience Store   1.0
2          Accessories Store   0.0
3  Middle Eastern Restaurant   0.0
4        Monument / Landmark   0.0


----Wexford, Maryvale----
                   venue  freq
0      Accessories Store   1.0
1  Vietnamese Restaurant   1.0
2             Smoke Shop   1.0
3          Shopping Mall   1.0
4         Sandwich Place   1.0


----Willowdale, Willowdale East----
                 venue  freq
0     Ramen Restaurant   3.0
1          Pizza Place   2.0
2  Japanese Restaurant   2.0
3     Sushi Restaurant   2.0
4          Coffee Shop   2.0


----Willowdale, Willowdale West----
           venue  freq
0    Pizza Place   1.0
1    Coffee Shop   1.0
2           Bank   1.0
3   Home Service   1.0
4  Grocery Store   1.0


----Woburn----
                 venue  freq
0          Coffee Shop   2.0
1         Soccer Field   1.0
2    Korean Restaurant   1.0
3    Convenience Store   1.0
4  Moroccan Restaurant   0

### Getting the top 10 venues in each Neighborhood

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
def get_neighborhoods_venues_sorted(toronto_grouped):
    num_top_venues = 10

    indicators = ['st', 'nd', 'rd']

    # create columns according to number of top venues
    columns = ['Neighborhood']
    for ind in np.arange(num_top_venues):
        try:
            columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
        except:
            columns.append('{}th Most Common Venue'.format(ind+1))

    # create a new dataframe
    neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
    neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

    for ind in np.arange(toronto_grouped.shape[0]):
        neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

    return neighborhoods_venues_sorted
get_neighborhoods_venues_sorted(toronto_grouped).head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
1,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Pharmacy,Pub,Sandwich Place,Coffee Shop,Airport Terminal,College Gym,Electronics Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Mobile Phone Shop,Convenience Store,Bridal Shop,Diner,Sandwich Place,Deli / Bodega,Restaurant,Middle Eastern Restaurant
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Restaurant,Sushi Restaurant,Thai Restaurant,Breakfast Spot,Juice Bar,Fast Food Restaurant,Butcher


### Defining functions for clusturing

Clusturing

In [27]:

# set number of clusters
kclusters = 5

def clusturing(toronto_grouped):
    toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

    # run k-means clustering
    kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

    toronto_merged = toronto_data
    # add clustering labels
    ns = get_neighborhoods_venues_sorted(toronto_grouped)
    ns.insert(0, 'Cluster Labels', kmeans.labels_)

    # merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
    toronto_merged = toronto_merged.join(ns.set_index('Neighborhood'), on='Neighborhood')

    toronto_merged.dropna(subset=['Cluster Labels'], inplace=True)
    return toronto_merged

Vizualizing

In [40]:
def viz(toronto_merged, cluster_names = None):
    # create map
    map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

    # set color scheme for the clusters
    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map
    markers_colors = []
    for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
        name = str(int(cluster))
        if cluster_names:
            name = cluster_names[name]
        else:
            name = 'Cluster '+name
        label = folium.Popup(str(poi) + ' \n ' + name, parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster-1)],
            fill=True,
            fill_color=rainbow[int(cluster-1)],
            fill_opacity=0.7).add_to(map_clusters)

    return map_clusters

Function to get the most commun venues in a cluster

In [29]:
def common_venues(df, toronto_grouped):
    print(df.shape[0], 'Neighborhood in the cluster')
    val = df.drop('Neighborhood', axis=1).to_numpy().flatten()
    unique_elements, counts_elements = np.unique(val, return_counts=True)
    nb = df['Neighborhood']
    col = np.concatenate([['Neighborhood'], unique_elements])
    freq = toronto_grouped.loc[toronto_grouped['Neighborhood'].isin(nb), col]
    freq.set_index('Neighborhood', inplace=True)
    freq = freq.transpose()
    # add a total
    freq['Total'] = freq.sum(axis=1)
    freq.sort_values(by='Total', ascending=False, inplace=True)
    return freq[['Total']]

### First with only torontos Borough

filter dataset, clusturing and vizualisation

In [30]:
# filter data set
n_to_keep = toronto_data.loc[toronto_data['Borough'].str.contains('Toronto'), 'Neighborhood']
toronto_grouped_loc = toronto_grouped.loc[toronto_grouped['Neighborhood'].isin(n_to_keep)]
toronto_grouped_loc.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
5,Berczy Park,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,2,0,0,0,0,0,1,1,0,2,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,2,0,0,0,0,0,3,4,0,0,0,0,0,0,0,1,0,1,0,0,0,

In [41]:
toronto_merged_loc = clusturing(toronto_grouped_loc)
viz(toronto_merged_loc)

Cluster analysis

In [32]:
for i in range(kclusters):
    print('cluster ', i, ':')
    c = toronto_merged_loc.loc[toronto_merged_loc['Cluster Labels'] == i, toronto_merged_loc.columns[[2] + list(range(6, toronto_merged_loc.shape[1]))]]
    print(common_venues(c, toronto_grouped_loc).head())

cluster  0 :
13 Neighborhood in the cluster
Neighborhood        Total
Coffee Shop            53
Café                   36
Restaurant             23
Italian Restaurant     15
Bakery                 14
cluster  1 :
4 Neighborhood in the cluster
Neighborhood  Total
Coffee Shop      46
Café             27
Hotel            23
Restaurant       18
Gym              13
cluster  2 :
1 Neighborhood in the cluster
Neighborhood     Total
Clothing Store      10
Coffee Shop          9
Bubble Tea Shop      3
Café                 3
Cosmetics Shop       3
cluster  3 :
3 Neighborhood in the cluster
Neighborhood         Total
Coffee Shop             34
Café                    11
Italian Restaurant      10
Hotel                    8
Japanese Restaurant      7
cluster  4 :
18 Neighborhood in the cluster
Neighborhood   Total
Park              16
Café              13
Coffee Shop       11
Restaurant         8
Grocery Store      8


Let's name the clusters

In [33]:
clusters_names = {'0': 'The place to eat whatever the hour', 
                '1': 'The business place',
                '2': 'The shopping area',
                '3': 'The world food',
                '4': 'The family place'}

Vizaualization with names

In [42]:
viz(toronto_merged_loc, clusters_names)

### Then with the whole scope

In [43]:
toronto_merged = clusturing(toronto_grouped)
viz(toronto_merged)

In [44]:
for i in range(kclusters):
    print('cluster ', i, ':')
    c = toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]
    print(common_venues(c, toronto_grouped).head())

cluster  0 :
69 Neighborhood in the cluster
Neighborhood    Total
Park               32
Coffee Shop        27
Pizza Place        20
Sandwich Place     16
Bank               15
cluster  1 :
6 Neighborhood in the cluster
Neighborhood  Total
Coffee Shop      69
Café             35
Hotel            30
Restaurant       24
Gym              17
cluster  2 :
5 Neighborhood in the cluster
Neighborhood         Total
Coffee Shop             39
Café                    12
Restaurant              10
Japanese Restaurant      8
Park                     7
cluster  3 :
2 Neighborhood in the cluster
Neighborhood          Total
Clothing Store           18
Coffee Shop              14
Fast Food Restaurant      6
Japanese Restaurant       6
Cosmetics Shop            4
cluster  4 :
16 Neighborhood in the cluster
Neighborhood        Total
Café                   41
Coffee Shop            36
Restaurant             20
Italian Restaurant     18
Pizza Place            16


All Neighborhood seems to have lots of coffee shops and café, other venues seems different ! <br>
Let's name those clusters

In [45]:
clusters_names = {'0': 'Everything you need is here', 
                '1': 'The business place',
                '2': 'The world food',
                '3': 'The shopping area',
                '4': 'The place to eat italian food'}

Display the final map

In [47]:
viz(toronto_merged, clusters_names)